# Tensorflow Object Detection API
- Tensorflow Object Detection API는 TensorFlow를 이용해서 Object Detection 모델을 train하고 deploy하는 것을 쉽게 도와주는 오픈소스 프레임워크.
- https://github.com/tensorflow/models/tree/master/research/object_detection
- Tutorial: https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/

# 전단계
- 구글드라이브 연결
- raw_data의 데이터압축파일을 VM local에 압축 푼다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 데이터셋 압축을 풀 디렉토리 생성.
!mkdir images

In [ ]:
from zipfile import ZipFile

with ZipFile('/content/drive/MyDrive/object_detection/sign_language_letters/raw_data/american_sign_language_letters.zip') as zipFile:
    zipFile.extractall('./images')

In [ ]:
# 리눅스 명령어로 풀기: 압축풀기 unzip 압축파일 -d 풀디렉토리
# !unzip -q /content/drive/MyDrive/object_detection/sign_language_letters/raw_data/american_sign_language_letters.zip -d /content/images

# Tensorflow Object Detection 2 API 설치
1. clone 
    - `!git clone https://github.com/tensorflow/models.git`
1. PYTHONPATH 환경설정에 models/research 추가  
1. 필요 모듈 설치
    - `!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk`
    - `!pip install -qq Cython contextlib2 pillow lxml matplotlib pycocotools`
1. proto 파일 컴파일
    - models/research 경로로 이동
        - `%cd models/research`
    - `!protoc object_detection/protos/*.proto --python_out=.`
1. setup.py 를 이용해 필요한 모듈 추가 설치
    - setup.py를 현재 디렉토리로 카피
        - `!cp object_detection/packages/tf2/setup.py . `
    - 설치
        - `!python -m pip install . `
    - 설치 확인 - 아래 스크립트 실행시 오류 없이 실행되면 설치 잘 된 것임.
        - `!python object_detection/builders/model_builder_tf2_test.py`
1. 원래 디렉토리로 이동
    - `%cd ../..`        

In [ ]:
# TFODA2 clone
!git clone https://github.com/tensorflow/models.git

In [ ]:
# models/research를 PYTHONPATH 환경설정 잡기
import os
os.environ["PYTHONPATH"] += ':/content/models/research'

In [ ]:
#필요 모듈 설치
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

In [ ]:
!pip install -qq Cython contextlib2 pillow lxml matplotlib pycocotools

In [ ]:
# .proto 파일들 컴파일
# models/research 로 이동후 protoc 실행
%cd models/research

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
# setup.py를 이용해서 추가 패키지들 설치
!cp object_detection/packages/tf2/setup.py .

In [ ]:
!python -m pip install .

In [ ]:
!pwd

In [ ]:
%cd ../..

In [ ]:
# 설치 성공 여부 테스트
!python models/research/object_detection/builders/model_builder_tf2_test.py

# 경로 설정
- 자주 사용되는 경로를 문자열 변수로 정의

In [ ]:
import os

BASE_PATH = "/content/drive/MyDrive/object_detection/sign_language_letters/workspace"
SCRIPT_PATH = "/content/drive/MyDrive/object_detection/sign_language_letters/scripts" #util 스크립트 파일들이 있는 경로
TF_OD_API_PATH = '/content/models' #TF object detection api2 설치 경로

IMAGE_PATH = '/content/images' #이미지, annotation 파일 압축푼 경로

LABEL_MAP_PATH = os.path.join(BASE_PATH, "labelmap") #라벨맵 설정파일 저장할 디렉토리
LABELMAP_FILE_PATH = os.path.join(LABEL_MAP_PATH, 'label_map.pbtxt') #라벨맵 파일 경로

#TFRecord 저장 디렉토리 - vm local
TF_RECORD_PATH = '/content/tfrecord'
os.makedirs(TF_RECORD_PATH, exist_ok=True) #디렉토리 생성

MODEL_PATH = os.path.join(BASE_PATH, 'model') #전이학습한 모델을 저장할 경로.(checkpoint-weights)
CHECK_POINT_PATH = os.path.join(MODEL_PATH, 'checkpoint') #학습한 weight 저장 경로
EXPORT_MODEL_PATH = os.path.join(MODEL_PATH, 'export_model') # 학습한 모델 저장 경로.
PIPELINE_CONFIG_PATH = os.path.join(MODEL_PATH, 'pipeline.config')#pipeline.config 설정파일 경로

# 다운받은 Pretrained 모델을 저장할 경로
PRE_TRAINED_MODEL_PATH = os.path.join(BASE_PATH, "pre_trained_model")

# Custom data 학습 시키기

## 다음 세가지 작업이 필요
<span style='font-weight:bold;font-size:1.3em'>1. Label Map 파일 생성</span>
- 분류 하고자 하는 object의 class와 그 class id 를 pbtxt text 파일로 작성
- `models\research\object_detection\data`

```
item {
  id: 1
  name: 'aeroplane'
}

item {
  id: 2
  name: 'bicycle'
}
...
```

<span style='font-weight:bold;font-size:1.3em'>2. pipeline.config</span>
- Model을 학습, 검증하기 위해 필요한 설정을 하는 파일
- `models\research\object_detection\samples\configs`

<span style='font-weight:bold;font-size:1.3em'>3. 학습/검증/테스트에 사용할 데이터셋을 TFRecord 로 구성</span>
- 주요 데이터셋을 TFRecord로 생성하는 코드
- `models\research\object_detection\dataset_tools`

# 설정파일 설정 및 데이터셋 준비

# Label Map 생성
- text(protocol buf 형식)
- 직접 text editor를 이용해 작성
- 코드로 생성

### Label A ~ Z 를 LabelMap에 작성

In [1]:
names = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
ids = range(1,27)
len(names), len(ids)

(26, 26)

In [ ]:
with open(LABELMAP_FILE_PATH, 'wt') as fw:
    for name, id in zip(names, ids):
        fw.write('item {\n')  # item {엔터
        fw.write("\tname:'{}'\n".format(name)) #탭 name='A'엔터
        fw.write("\tid:{}\n".format(id)) #탭id=1엔터
        fw.write("}\n") # }엔터

# TFRecord 생성
- scripts/generate_tfrecord.py 사용
- 옵션(command line argumnets)
    - -x (--xml_dir) : annotation 파일들이 있는 경로
    - -l (--labels_path) : LabelMap 파일 경로
    - -o (--output_path) : tfrecord 파일 저장할 경로

In [ ]:
# train dataset의 tfrecord생성
f"!python {SCRIPT_PATH}/generate_tfrecord.py -x {IMAGE_PATH}/train  -l {LABELMAP_FILE_PATH} -o {TF_RECORD_PATH}/train.tfr"

In [ ]:
# test set tfrecord 생성
f"!python {SCRIPT_PATH}/generate_tfrecord.py -x {IMAGE_PATH}/test  -l {LABELMAP_FILE_PATH} -o {TF_RECORD_PATH}/test.tfr"

In [ ]:
# validation set tfrecord생성
f"!python {SCRIPT_PATH}/generate_tfrecord.py -x {IMAGE_PATH}/valid  -l {LABELMAP_FILE_PATH} -o {TF_RECORD_PATH}/valid.tfr"

In [ ]:
# tfrecord파일들을 google drive에 backup (복사)

In [ ]:
!cp /content/tfrecord/*.tfr  /content/drive/MyDrive/object_detection/sign_language_letters/workspace/tfrecord

In [ ]:
# 구글드라이브의 tfrecord파일을 local vm으로 restore 하기
!cp /content/drive/MyDrive/object_detection/sign_language_letters/workspace/tfrecord/*.tfr   /content/tfrecord

# Pretrained Model Download
- Tensorflow object detection API는 MS COCO 2017 dataset으로 미리 학습시킨 다양한 Object Detection 모델을 제공한다.
- tf2 detection Model Zoo: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md
- SSD MobileNet V2 FPNLite 320x320 다운로드
    - 성능은 떨어지지만 학습속도가 빠르다.

In [ ]:
# wget url  : url의 파일을 다운로드
!wget  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

In [ ]:
# workspace/pre_trained_model 로 옮긴뒤 압축 풀기.
# 옮기기. mv 옮길파일   옮길디렉토리
f"!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRE_TRAINED_MODEL_PATH}"

In [ ]:
# 압축풀기  tar -zxvf 앞축파일 -C 압축풀디렉토리
f"!tar -zxvf  {PRE_TRAINED_MODEL_PATH}/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz  \
-C  {PRE_TRAINED_MODEL_PATH}"

# Pipeline.config 설정 변경

## pipeline.config  파일 개요
- Model을 학습, 검증하기 위해 필요한 설정을 하는 파일
- 구조
    - https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/configuring_jobs.md
    - **model**
        - 사용하는 모델에 대한 설정
        - class 개수
        - 입력이미지 size
        - anchor 설정
    - **train_config**
        - Train(학습)관련 설정
        - batch_size
            - 사용하는 GPU의 메모리 크기에 맞게 조절한다.
        - image augmentation관련 설정 등
        - optimizer관련 설정
        - 학습에 사용할 weight 파일의 경로
    - **train_input_reader**
        - labelmap 파일 경로
        - train tfrecord 파일 경로
    - **eval_config**
        - evaluation(평가)을 위해 사용하는 metric 설정
    - **eval_input_reader**
        - labelmap 파일 경로
        - evaluation tfreord 파일 경로
        

## Pretrain model의 pipeline.config 파일 카피
- pretrained 모델의 압축을 풀면 pipeline.config 파일이 있다.
- workspace\model 로 copy 한다.

In [ ]:
f'!cp {os.path.join(PRE_TRAINED_MODEL_PATH, "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8", "pipeline.config")} \
 {PIPELINE_CONFIG_PATH}'

## pipeline.config 설정 변경
- pipeline.config 내용 변경은 파일을 **직접 변경**할 수도 있고 **코드상에서 변경**할 수도 있다.

### 필수 변경사항
-  class개수 변경
-  train 배치 사이즈 변경 - gpu 메모리 사양에 맞게 변경한다.
-  pretrained model CHECKPOINT가 저장된 경로 설정
-  pretrained model이 어떤 종류의 모델인지 설정
-  train 관련 변경
    -  labelmap 파일 경로 설정
    -  train 용 tfrecord 파일 경로 지정
-  evaluation 관련 변경
    -  labelmap 파일 경로 설정
    -  evaluation 용 tfrecord 파일 경로 지정

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
# pipeline.config 파일을 읽어서 확인(수정과 상관없이 내용확인)
config = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG_PATH)
print(type(config))
config

In [ ]:
# 빈 pipeline.config를 생성
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
print(type(pipeline_config))
print(pipeline_config)

In [ ]:
# pipeline.config 를 text로 읽어서 빈 pipeline.config에 넣는다.
with tf.io.gfile.GFile(PIPELINE_CONFIG_PATH, 'r') as fr: 
    proto_str = fr.read()
    text_format.Merge(proto_str, pipeline_config)

In [ ]:
print(pipeline_config)

In [ ]:
# 설정 값 변경
# 설정 계층구조에 맞게  . 표기법을 이용해 접근한다. 

# 검출해야 하는 object 개수를 변경
pipeline_config.model.ssd.num_classes = 26 
# 배치 size를 변경 - gpu 메모리에 맞춰서 최대한 크게 잡아준다.
pipeline_config.train_config.batch_size = 8
# Pretrained 모델의 checkpoint 파일(weight) 경로 지정 
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(PRE_TRAINED_MODEL_PATH, "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8", "checkpoint", "ckpt-0")
# Pretrained 모델이 어떤 종류를 학습한 모델인지 설정
pipeline_config.train_config.fine_tune_checkpoint_type = 'detection' #object detection

# train_input_reader 설정
# LabelMap 파일 경로 설정
pipeline_config.train_input_reader.label_map_path = LABELMAP_FILE_PATH
# train.tfr 경로 설정
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(TF_RECORD_PATH, 'train.tfr')]

## eval_input_reader 설정
# Label Map 파일 경로 설정
pipeline_config.eval_input_reader[0].label_map_path = LABELMAP_FILE_PATH
# test.tfr 경로 설정
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(TF_RECORD_PATH, 'test.tfr')]

In [ ]:
# TrainEvalPipelineConfig의 변경된 설정을 파일에 쓰기
config_txt = text_format.MessageToString(pipeline_config) #TrainEvalPipelineConfig객체의 내용을 string으로 변환
print(type(config_txt))
print(config_txt)
with tf.io.gfile.GFile(PIPELINE_CONFIG_PATH, "wb") as fw:
    fw.write(config_txt)

# Model 학습
- 다음 명령어를 실행한다.
> - 개인컴퓨터에서 할 경우 시간이 오래 걸리므로 terminal에서 실행한다.


```
python models/research/object_detection/model_main_tf2.py --model_dir=workspace/model/checkpoint --pipeline_config_path=workspace/model/pipeline.config --num_train_steps=3000
```

## 옵션
- model_dir: 학습한 모델의 checkpoint 파일을 저장할 경로. (1000 step당 저장한다.)
- pipeline_config_path: pipeline.config 파일 경로
- num_train_steps: 학습할 step 수

In [ ]:
f'!python {os.path.join(TF_OD_API_PATH, "research","object_detection", "model_main_tf2.py")}\
  --model_dir={CHECK_POINT_PATH}\
  --pipeline_config_path={PIPELINE_CONFIG_PATH}\
  --num_train_steps=1000'

# 학습한 모델 추출(export)
- `models/research/object_detection/exporter_main_v2.py` 사용
- 옵션
    - `exporter_main_v2.py --helpshort || exporter_main_v2.py --helpfull`
    - input_type : input node type
        - image_tensor, encoded_image_string_tensor
    - train_checkpoint: 학습된 checkpoint 파일이 저장된 경로(folder/directory)
    - pipeline_config_path: pipeline.config 파일의 경로 (파일명 포함)
    - output_directory: export된 모델을 저장할 경로.
- 추출된 디렉토리 구조
```bash
output_dir
├─ checkpoint/
├─ saved_model/
└─ pipeline.config
```
    - checkpoint: custom data 학습한 checkpoint 파일들을 이 디렉토리로 복사한다.
    - save_model: pipeline.config 설정에 맞춰 생성된 model
    - pipeline.config: pipeline.config 설정파일

In [ ]:
f"!python {os.path.join(TF_OD_API_PATH, 'research', 'object_detection', 'exporter_main_v2.py')} \
--input_type=image_tensor  --trained_checkpoint_dir={CHECK_POINT_PATH}  \
--pipeline_config_path={PIPELINE_CONFIG_PATH} --output_directory={EXPORT_MODEL_PATH}"

# Inference(추론)

### 사용 함수,메소드
-  ### tf.convert_to_tensor(array_like, dtype)
    - array_like 를 Tensoflow Tensor 객체로 변환
    - `tf.convert_to_tensor([[1,2],[3,4]])`
- ### detection_model.preprocess(image 4차원 ndarray)
    - 전달받은 이미지를 model의 input shape에 맞게 resizing 한다.
    - 반환값: (resize된 image Tensor, 이미지의 shape) 을 tuple로 반환
- ### detection_model.predict(image tensor, image_shape tensor)
    - 추론/detection 메소드
    - 이미지와 image shape을 받아서 detection한 결과를 딕셔너리로 반환한다.
    - **반환 dictionary key**
        - **preprocessed_inputs**:  입력 이미지 Tensor. preprocess()로 처리된 이미지. 
        - **feature_maps**: List. feature map 들을 반환
        - **anchors**: 2D Tensor. normalize 된 anchor box들의 좌표를 반환. 2-D float tensor: \[num_anchors, 4\]
        - **final_anchors**: 3D Tensor. batch 당 anchors. (anchors에 batch가 포함된 것). \[batch_size, num_anchors, 4\]
        - **box_encodings**: 3D float tensor. predict한 box들의 normalize된 좌표. \[batch_size, num_anchors,box_code_dimension\]
        - **class_predictions_with_background**: 3D Tensor. 클래스 확률을 반환.(logit). \[batch_size, num_anchors, num_classes+1]\
            - background 확률을 포함해서 num_classes+1개가 된다. (index 0: background)
            
- ### detection_model.postprocess(prediction_dict, shape)
    - predict()가 예측한 결과에서 **Non-Maxinum Suppression**을 실행해서 최종 Detection 결과를 반환한다.
        - predict()는 anchor별로 예측결과를 모아서 주고 post-process는 최종 결과를 추출해서 반환.
    - **반환 dictionary key**
        - **num_detections**: Detect한 개수 (bounding box 개수)
        - **detection_boxes**: [batch, max_detections, 4]. 후처리한 detection box
        - **detection_scores**: [batch, max_detections]. post-processed detection box들의 detection score들 (detection score는 box안에 물체가 있을 확률값 - confidence score).
        - **detection_classes**: [batch, max_detections] tensor with classes for post-processed detection classes.
        - **raw_detection_boxes**:[batch, total_detections, 4] Non-Max Suppression 하기 전의 감지된 box들
        - **raw_detection_scores**: [batch, total_detections, num_classes_with_background]. raw detection box들의 class별 점수
        - **detection_multiclass_scores**: [batch, max_detections, num_classes_with_background] post-processed이후 남은 bounding box 들의 class별 점수. LabelMap의 class에 background가 추가되어 계산된다.
        - **detection_anchor_indices**: [batch, max_detections] post-processed 이후 나은 anchor box의 index들.

In [ ]:
import os 
import cv2
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from object_detection.utils import label_map_util # labelmap의 내용을 읽어오는 함수
from object_detection.utils import visualization_utils as viz_utils #결과를 이미지에 출력하는 모듈
from object_detection.utils import config_util #Pipeline.config loading
from object_detection.builders import model_builder # pipeline.config를 이용해 모델 생성.

In [ ]:
!unzip -q /content/drive/MyDrive/object_detection_src/sign_language_letters/workspace/model/checkpoint2/sign_lang_50000_checkpoint.zip  -d  /content/drive/MyDrive/object_detection_src/sign_language_letters/workspace/model/checkpoint2

In [ ]:
CHECK_POINT_PATH = "/content/drive/MyDrive/object_detection/sign_language_letters/workspace/model/checkpoint"

# pipeline.config를 이용해 model을 build (생성)

# pipeline.config 읽어오기
configs = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG_PATH)

# config를 이용해서 모델을 생성
detection_model = model_builder.build(model_config=configs['model'], is_training=False)
print(type(detection_model))

# 생성된 모델의 weight들을 학습한 weight들로 덮어쓰기.
# 학습된 checkpoint(weight)를 loading
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECK_POINT_PATH, 'ckpt-2')).expect_partial()

In [ ]:
# 추론하는 함수
# tensorflow 모델의 추론 속도를 빠르게 해 주는 decorator
@tf.function  
def detect_func(image):
    """
    image(tensor)를 받아서 추론 후 그 결과를 반환하는 함수
    1. preprocessing
    2. predict 
    3. postprocessing -> 이결과를 반환
    """
    image, shape = detection_model.preprocess(image) # input_shape으로 resize, 정규화: -1 ~ 1
    pred = detection_model.predict(image, shape) #추론
    result = detection_model.postprocess(pred, shape) #후처리한 최종 결과 반환
    return result


In [ ]:
# image 로딩 -> Tensor 변환 - detect_func()이용해서 추론
filepath = 'a.jpg'

image_np = cv2.cvtColor(cv2.imread(filepath) , cv2.COLOR_BGR2RGB)
input_tensor = tf.convert_to_tensor(image_np[np.newaxis,...], dtype=tf.float32)
print(input_tensor.shape, input_tensor.dtype)
pred = detect_func(input_tensor)

In [ ]:
num_detections = int(pred.pop('num_detections'))
num_detections

In [ ]:
detections = {key: value[0, :num_detections].numpy()  for key, value in pred.items()}

detections['num_detections'] = num_detections

In [ ]:
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

In [ ]:
# labelmap.pbtxt 파일의 내용을 읽어오기.
category_index = label_map_util.create_category_index_from_labelmap(LABELMAP_FILE_PATH)
print(type(category_index))
category_index

In [ ]:
# 검출한 결과를 원본 영상에 그리기 (bounding box, class label)
MIN_CONF_THRESHOLD = 0.5 #detection_scores(confidence score) 가 지정한 값 이상인 bounding box만 그리기.
img = viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np, # 원본 이미지
        detections['detection_boxes'], # bounding box 좌표
        detections['detection_classes'] + 1, # label
        detections['detection_scores'], # confidence score
        category_index, #label map 
        use_normalized_coordinates=True, # bounding box 좌표가 normalize됬는지 여부
        max_boxes_to_draw=5, #이미지위에 최대 몇개의 bounding box를 그릴지
        min_score_thresh = MIN_CONF_THRESHOLD # confidence score가 지정한 값 이상인 것만 그리기.
)

In [ ]:
type(img), img.shape

In [ ]:
%matplotlib inline

In [ ]:
save_file_path = 'n_detect.jpg'
plt.figure(figsize=(10,10))
plt.imshow(img)
plt.savefig(save_file_path) #저장
plt.show()

In [ ]:
detections['detection_scores']

In [ ]:
detections['detection_classes']